# Spark SQL

https://spark.apache.org/sql/

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark_sql").getOrCreate()
df_flights_parquet = (spark.read
        .format("parquet")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("data/flights/summary-data/parquet/*"))

In [ ]:
df_flights_parquet

In [ ]:
for t in spark.catalog.listTables():
    print(t.name, t.tableType)

### Rejestracja jako tabela tymczasowa (temp view)

In [ ]:
# Rejestracja DataFrame jako tymczasowej tabeli SQL
df_flights_parquet.createOrReplaceTempView("flights")

In [ ]:
for t in spark.catalog.listTables():
    print(t.name, t.tableType)

In [ ]:
spark.sql("Describe formatted flights").show()

In [ ]:
spark.sql("SELECT * FROM flights LIMIT 10").show()


In [ ]:
spark.sql("SELECT COUNT(*) AS liczba_lotow FROM flights").show()


In [ ]:
spark.sql("""
    SELECT ORIGIN_COUNTRY_NAME, COUNT(*) AS liczba_lotow
    FROM flights
    GROUP BY ORIGIN_COUNTRY_NAME
    ORDER BY liczba_lotow DESC
""").show()


In [ ]:
spark.sql("""
    SELECT *
    FROM flights
    WHERE origin_country_name = 'United States'
    LIMIT 10
""").show()

### Tworzenie trwałej tabeli (managed table) na podstawie DataFrame

**Cechy:**
- Dane są przechowywane i zarządzane przez Spark w domyślnej lokalizacji:
np. spark-warehouse/moja_baza.db/nazwa_tabeli/

- Usunięcie tabeli (DROP TABLE) usuwa również dane z dysku.

- Spark zarządza metadanymi i fizycznymi plikami danych.

`SQL_syntax`

```
CREATE TABLE flights (
  id INT,
  origin STRING
)
USING parquet;

```

Aby zapisać DataFrame jako trwałą tabelę Spark SQL, możesz użyć metody **.write.saveAsTable()**
To tworzy trwałą tabelę o nazwie `flights_managed` w domyślnej bazie `(default)`.

In [ ]:
df_flights_parquet.write.mode("overwrite").saveAsTable("flights_managed")

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS moja_baza")

In [ ]:
df_flights_parquet.write.mode("overwrite").saveAsTable("moja_baza.flights_managed")

In [ ]:

spark.sql("DESCRIBE FORMATTED flights_managed").show(truncate=False)

In [ ]:
spark.sql("SELECT * FROM flights LIMIT 10").show()

In [ ]:
spark.sql("""
    SELECT ORIGIN_COUNTRY_NAME, COUNT(*) AS liczba_lotow
    FROM flights
    GROUP BY ORIGIN_COUNTRY_NAME
    ORDER BY liczba_lotow DESC
""").show()

In [ ]:
spark.catalog.currentDatabase()

In [ ]:
# tworzenie df odwołująć się do tabeli
df = spark.table("flights_managed ")

In [ ]:
df.show()

### Tworzenie niezarządzanych tabeli (External Table)

**Cechy:**
- Dane przechowywane są w podanej lokalizacji (np. HDFS, S3, lokalny katalog).

- Spark zarządza tylko **metadanymi**, a nie plikami.

- Usunięcie tabeli (DROP TABLE) **nie usuwa danych**, tylko metadane z metastore.

- Idealna, gdy:

    - Dane są współdzielone z innymi systemami

    - Nie chcesz, by Spark zarządzał plikami
 

`SQL_syntax`
```
CREATE EXTERNAL TABLE flights_ext (
  id INT,
  origin STRING
)
USING parquet
LOCATION '/ścieżka/do/danych';


```

In [ ]:
df_flights_parquet.write.mode("overwrite").option("path", "flights_external").saveAsTable("flights_ext")

In [ ]:
spark.sql("DESCRIBE FORMATTED flights_ext").show(truncate=False)

In [ ]:
for t in spark.catalog.listTables():
    print(t.name, t.tableType)

In [ ]:
spark.sql("SELECT * FROM flights_ext LIMIT 10").show()

In [ ]:
spark.sql("""
    SELECT ORIGIN_COUNTRY_NAME, COUNT(*) AS liczba_lotow
    FROM flights_ext
    GROUP BY ORIGIN_COUNTRY_NAME
    ORDER BY liczba_lotow DESC
""").show()

| Metoda                            | Co robi                        |
| --------------------------------- | ------------------------------ |
| `createOrReplaceTempView("name")` | Tymczasowy widok (sesyjny)     |
| `write.saveAsTable("name")`       | Trwała tabela w Hive metastore |


In [ ]:
spark.sql("""
    DROP table flights_ext
""").show()

In [ ]:
spark.sql("""
    DROP table flights_managed
""").show()

In [ ]:
from pyspark.sql.functions import spark_partition_id
print(spark_partition_id())

### Spark.catalog

`spark.catalog` to obiekt klasy Catalog, który umożliwia:

- Przeglądanie baz danych i tabel

- Zarządzanie widokami tymczasowymi

- Odpytywanie o funkcje zarejestrowane w Spark SQL

| Metoda                       | Opis                                               | Przykład                                        |
| ---------------------------- | -------------------------------------------------- | ----------------------------------------------- |
| `listDatabases()`            | Zwraca listę baz danych                            | `spark.catalog.listDatabases()`                 |
| `listTables(dbName=None)`    | Zwraca listę tabel w bazie (domyślnie w aktualnej) | `spark.catalog.listTables("moja_baza")`         |
| `listFunctions(dbName=None)` | Zwraca listę funkcji w bazie                       | `spark.catalog.listFunctions()`                 |
| `currentDatabase()`          | Zwraca nazwę aktualnie używanej bazy               | `spark.catalog.currentDatabase()`               |
| `setCurrentDatabase(dbName)` | Ustawia aktywną bazę                               | `spark.catalog.setCurrentDatabase("moja_baza")` |
| `isCached(tableName)`        | Sprawdza, czy tabela jest zbuforowana (cached)     | `spark.catalog.isCached("people")`              |
| `cacheTable(tableName)`      | Buforuje tabelę w pamięci                          | `spark.catalog.cacheTable("people")`            |
| `uncacheTable(tableName)`    | Usuwa buforowanie tabeli                           | `spark.catalog.uncacheTable("people")`          |
| `clearCache()`               | Czyści cały cache Spark                            | `spark.catalog.clearCache()`                    |


In [ ]:
spark.catalog.listDatabases()

In [ ]:
# tworzenie bazy danych
spark.sql("CREATE DATABASE IF NOT EXISTS moja_baza")

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.catalog.listTables("default")

In [ ]:
spark.catalog.listTables("moja_baza")

In [ ]:
spark.catalog.cacheTable("flights")

In [ ]:
spark.sql("SELECT * FROM flights LIMIT 10").show()

In [ ]:
spark.catalog.cacheTable("flights_managed")

In [ ]:
spark.sql("SELECT * FROM flights_managed LIMIT 10").show()

In [ ]:
spark.catalog.cacheTable("flights_ext")

In [ ]:
spark.sql("SELECT * FROM flights_ext LIMIT 10").show()

In [ ]:
spark.sql("""
    DROP table flights_ext
""").show()

In [ ]:
spark.sql("""
    DROP table flights_managed
""").show()

In [ ]:
spark.sql("""
    DROP database moja_baza
""").show()